In [1]:
# This project is going to look at warding data from worlds 2017
# This does nto include the play-in games 
# The goal here is to see who wards the least, how their warding changes
# When they are winning vs. losing 
# Lastly do they ward differently when they are ahead in lane vs behind
import pandas as pd
import numpy as np

In [2]:
# Reading in worlds data 

raw_worlds_data = pd.read_csv('./2017-worlds-match-data-OraclesElixir-2017-11-05.csv')
raw_worlds_data.head()

,gameid,url,league,split,date,week,game,patchno,playerid,side,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
0,1002360213,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43001.00184,1.1,1,7.18,1,Blue,...,6.063604,3349,3153,196,4880,4788,92,5025,4556,469
1,1002360213,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43001.00184,1.1,1,7.18,2,Blue,...,3.625442,2956,2667,289,4323,4234,89,3318,3332,-14
2,1002360213,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43001.00184,1.1,1,7.18,3,Blue,...,9.222615,3140,3875,-735,5147,5799,-652,4592,5083,-491
3,1002360213,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43001.00184,1.1,1,7.18,4,Blue,...,9.180212,3142,3230,-88,5372,5176,196,3112,3081,31
4,1002360213,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43001.00184,1.1,1,7.18,5,Blue,...,0.318021,2533,2051,482,4256,3300,956,3053,2882,171


In [3]:
# Lets remove all games that start with week < 2.0 or that have QF, SF,
# and F

# Checking if date is numeric if so we can select by it 
raw_worlds_data['date'].dtypes

# The games we want range from 43013.07499 - 43027.11118
# lets make a mask that only includes that range
games_mask = (raw_worlds_data['date'] >= 43013.07499) & (raw_worlds_data['date'] <43027.11118)
corrected_dates = raw_worlds_data[games_mask]
corrected_dates.tail()


,gameid,url,league,split,date,week,game,patchno,playerid,side,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
1075,1002380500,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43023.15065,3.4,6,7.18,8,Red,...,8.165217,3624,3231,393,5739,5226,513,4832,4720,112
1076,1002380500,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43023.15065,3.4,6,7.18,9,Red,...,11.191304,3850,2871,979,6348,4373,1975,2860,3205,-345
1077,1002380500,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43023.15065,3.4,6,7.18,10,Red,...,0.208696,2535,2318,217,3789,3509,280,2735,2515,220
1078,1002380500,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43023.15065,3.4,6,7.18,100,Blue,...,31.852174,14561,16999,-2438,22809,26180,-3371,18115,18127,-12
1079,1002380500,http://matchhistory.na.leagueoflegends.com/en/...,WC,2017-W,43023.15065,3.4,6,7.18,200,Red,...,29.582609,16999,14561,2438,26180,22809,3371,18127,18115,12


In [4]:
# Lets start removing unwanted data. First we dont need the cols 1-9 so lets remove those
corrected_dates = corrected_dates.drop(corrected_dates.columns[1:10],axis =1)
corrected_dates.head(10)

,gameid,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
468,1002390182,Top,Letme,Royal Never Give Up,Cho'gath,Gragas,Xayah,Alistar,Taliyah,Lucian,...,8.748387,3114,3225,-111,4986,4903,83,4493,4817,-324
469,1002390182,Jungle,Mlxg,Royal Never Give Up,Rek'Sai,Gragas,Xayah,Alistar,Taliyah,Lucian,...,4.748387,2944,3560,-616,5052,5266,-214,3378,3686,-308
470,1002390182,Middle,Xiaohu,Royal Never Give Up,Ryze,Gragas,Xayah,Alistar,Taliyah,Lucian,...,9.341935,3113,3623,-510,5275,5322,-47,4478,5053,-575
471,1002390182,ADC,Uzi,Royal Never Give Up,Kog'Maw,Gragas,Xayah,Alistar,Taliyah,Lucian,...,10.348387,3035,3205,-170,4884,5105,-221,3192,3336,-144
472,1002390182,Support,Ming,Royal Never Give Up,Janna,Gragas,Xayah,Alistar,Taliyah,Lucian,...,0.258065,2279,2307,-28,3528,3403,125,2742,2795,-53
473,1002390182,Top,Thaldrin,Fenerbahce Esports,Maokai,Kalista,Jarvan IV,Sejuani,Shen,Galio,...,8.051613,3225,3114,111,4903,4986,-83,4817,4493,324
474,1002390182,Jungle,Crash,Fenerbahce Esports,Nidalee,Kalista,Jarvan IV,Sejuani,Shen,Galio,...,5.651613,3560,2944,616,5266,5052,214,3686,3378,308
475,1002390182,Middle,Frozen,Fenerbahce Esports,Syndra,Kalista,Jarvan IV,Sejuani,Shen,Galio,...,10.322581,3623,3113,510,5322,5275,47,5053,4478,575
476,1002390182,ADC,Padden,Fenerbahce Esports,Tristana,Kalista,Jarvan IV,Sejuani,Shen,Galio,...,10.658065,3205,3035,170,5105,4884,221,3336,3192,144
477,1002390182,Support,Japone,Fenerbahce Esports,Lulu,Kalista,Jarvan IV,Sejuani,Shen,Galio,...,1.264516,2307,2279,28,3403,3528,-125,2795,2742,53


In [5]:
# There exists some team data so lets go ahead and remove that
player_mask = corrected_dates['player'] != 'Team'
player_data = corrected_dates[player_mask]
player_data.head()

,gameid,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
468,1002390182,Top,Letme,Royal Never Give Up,Cho'gath,Gragas,Xayah,Alistar,Taliyah,Lucian,...,8.748387,3114,3225,-111,4986,4903,83,4493,4817,-324
469,1002390182,Jungle,Mlxg,Royal Never Give Up,Rek'Sai,Gragas,Xayah,Alistar,Taliyah,Lucian,...,4.748387,2944,3560,-616,5052,5266,-214,3378,3686,-308
470,1002390182,Middle,Xiaohu,Royal Never Give Up,Ryze,Gragas,Xayah,Alistar,Taliyah,Lucian,...,9.341935,3113,3623,-510,5275,5322,-47,4478,5053,-575
471,1002390182,ADC,Uzi,Royal Never Give Up,Kog'Maw,Gragas,Xayah,Alistar,Taliyah,Lucian,...,10.348387,3035,3205,-170,4884,5105,-221,3192,3336,-144
472,1002390182,Support,Ming,Royal Never Give Up,Janna,Gragas,Xayah,Alistar,Taliyah,Lucian,...,0.258065,2279,2307,-28,3528,3403,125,2742,2795,-53


In [6]:
# Now we have just the player data lets make sure any of it is not null
null_player = player_data.isnull().any()
null_player[null_player == True]


herald        True
heraldtime    True
fbaron        True
fbarontime    True
dtype: bool

In [7]:
# There are 4 cols with null values, It looks like these are blank for
# If the obj was not taken
# Going to substitute -1 for not taken
player_data = player_data.fillna(value = -1)

# Test agian to make sure
null_player = player_data.isnull().any()
null_player[null_player == True]


Series([], dtype: bool)

In [8]:
# The series is empty so we filled the null values with -1 correctly
# Now lets output this so we have it saved for later use. 
# Using index = False to remove the indexing in the file
player_data.to_csv('Player_specific_data.csv', index=False, encoding = 'utf-8')

In [9]:
# Lets take a very high overview of the data
player_data.describe()

,gameid,gamelength,result,k,d,a,teamkills,teamdeaths,fb,fbassist,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
count,5.100000e+02,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,510.000000,...,510.000000,510.000000,510.000000,510.000000,510.00000,510.00000,510.000000,510.000000,510.000000,510.000000
mean,1.002385e+09,35.899020,0.500000,1.845098,1.847059,4.888235,9.225490,9.235294,0.100000,0.188235,...,6.430154,3056.517647,3056.517647,0.000000,4821.77451,4821.77451,0.000000,3645.949020,3645.949020,0.000000
std,5.011083e+03,9.025928,0.500491,2.035424,1.503154,3.826962,5.351875,5.368863,0.300295,0.391284,...,3.531491,449.205650,449.205650,413.785296,760.89093,760.89093,724.620323,843.318465,843.318465,402.480755
min,1.002380e+09,20.866667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.047923,1744.000000,1744.000000,-1823.000000,2629.00000,2629.00000,-2639.000000,1664.000000,1664.000000,-1237.000000
25%,1.002380e+09,27.566667,0.000000,0.000000,1.000000,2.000000,5.000000,5.000000,0.000000,0.000000,...,3.670284,2758.500000,2758.500000,-230.500000,4312.25000,4312.25000,-415.750000,2973.500000,2973.500000,-243.000000
50%,1.002380e+09,34.300000,0.500000,1.000000,2.000000,4.000000,9.000000,9.000000,0.000000,0.000000,...,7.746694,3131.000000,3131.000000,0.000000,4911.00000,4911.00000,0.000000,3361.500000,3361.500000,0.000000
75%,1.002390e+09,44.066667,1.000000,3.000000,3.000000,7.000000,12.000000,12.000000,0.000000,0.000000,...,9.386723,3354.750000,3354.750000,230.500000,5298.00000,5298.00000,415.750000,4488.750000,4488.750000,243.000000
max,1.002391e+09,53.983333,1.000000,11.000000,8.000000,22.000000,24.000000,24.000000,1.000000,1.000000,...,11.936967,4721.000000,4721.000000,1823.000000,7161.00000,7161.00000,2639.000000,5327.000000,5327.000000,1237.000000


In [10]:
# Lets take a look at the correlation of variables 
player_corr = player_data.corr()
player_corr.head()

,gameid,gamelength,result,k,d,a,teamkills,teamdeaths,fb,fbassist,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
gameid,1.000000,5.361851e-03,0.000000e+00,-0.075439,-0.103318,-0.117939,-0.143456,-0.144632,-5.937086e-19,0.025376,...,0.035564,0.030726,0.030726,0.000000e+00,0.003723,0.003723,0.000000e+00,0.015879,0.015879,0.000000e+00
gamelength,0.005362,1.000000e+00,1.545092e-18,0.146437,0.196631,0.292241,0.278465,0.275260,4.224861e-18,0.007590,...,-0.009645,-0.006110,-0.006110,-1.853904e-18,-0.023513,-0.023513,-5.463991e-18,-0.025894,-0.025894,-2.028941e-18
result,0.000000,1.545092e-18,1.000000e+00,0.342318,-0.464839,0.544148,0.650951,-0.650719,-6.535948e-03,-0.040129,...,0.032717,0.050937,-0.050937,1.105949e-01,0.134471,-0.134471,2.824045e-01,0.021589,-0.021589,9.046938e-02
k,-0.075439,1.464373e-01,3.423177e-01,1.000000,-0.102152,0.065115,0.525873,-0.086189,1.861051e-01,-0.059523,...,0.449189,0.442905,0.297408,1.579517e-01,0.520225,0.261905,2.712510e-01,0.125203,0.075540,1.040571e-01
d,-0.103318,1.966308e-01,-4.648391e-01,-0.102152,1.000000,-0.129001,-0.117080,0.714347,-8.704852e-04,-0.034464,...,-0.069846,-0.061659,0.076524,-1.500114e-01,-0.127019,0.141887,-2.823665e-01,-0.067263,-0.002859,-1.349451e-01


In [11]:
# There is a lot of data there and some of it really messy. 
# Lets subselect the data for things that are highly correlated
# I.E. >0.5 and <-0.5
pos_corr = player_corr >= 0.5
neg_corr = player_corr <= -0.5

In [12]:
# Lets send the highly corr values to a csv file for later viewing
player_corr = player_corr[neg_corr | pos_corr]

player_corr.to_csv('Player_highly_corr.csv', encoding = 'utf-8')


In [13]:
# Lets see what is highly corr with wards per min
player_corr['wpm'].sort_values(axis = 0, ascending = False).dropna()

wpm                     1.000000
wardshare               0.962527
wards                   0.907691
visionwardbuys          0.826563
visionwards             0.818416
oppxpat10              -0.509275
xpat10                 -0.511537
dmgtochamps            -0.512330
earnedgpm              -0.526030
goldat10               -0.612470
oppgoldat10            -0.616669
dmgtochampsperminute   -0.627994
goldat15               -0.646841
oppgoldat15            -0.676293
minionkills            -0.734666
cspm                   -0.834138
Name: wpm, dtype: float64

In [14]:
# Here we see some interesting correlations CSPM is the most neg corr
# This indicates that as CSPM goes up WPM goes down, same is true for
# Minion kills, oppgoldat15, goldat15, dmg to enempy champs per min
# Lets plot some of these data points
import matplotlib.pyplot as plt

player_data['wpm'].describe()


count    510.000000
mean       0.687687
std        0.341742
min        0.127011
25%        0.437363
50%        0.572114
75%        0.926544
max        1.853088
Name: wpm, dtype: float64

In [15]:
# Lets take a look at those players that place <= 25% of wards 
# and see there CSPM
wpm_filter = player_data['wpm'] <= 0.437363
wpm_v_csm = player_data[wpm_filter]
wpm_v_csm.head()

,gameid,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
471,1002390182,ADC,Uzi,Royal Never Give Up,Kog'Maw,Gragas,Xayah,Alistar,Taliyah,Lucian,...,10.348387,3035,3205,-170,4884,5105,-221,3192,3336,-144
475,1002390182,Middle,Frozen,Fenerbahce Esports,Syndra,Kalista,Jarvan IV,Sejuani,Shen,Galio,...,10.322581,3623,3113,510,5322,5275,47,5053,4478,575
476,1002390182,ADC,Padden,Fenerbahce Esports,Tristana,Kalista,Jarvan IV,Sejuani,Shen,Galio,...,10.658065,3205,3035,170,5105,4884,221,3336,3192,144
480,1002390191,Top,Expect,G2 Esports,Shen,Jarvan IV,Janna,Lulu,Ryze,Syndra,...,9.150090,3332,3046,286,4888,4874,14,4258,4200,58
482,1002390191,Middle,Perkz,G2 Esports,Corki,Jarvan IV,Janna,Lulu,Ryze,Syndra,...,10.198915,3418,3389,29,5157,5173,-16,4758,5040,-282


In [16]:
# Lets group these by player so we only have unique values 
wpm_grouped = wpm_v_csm.groupby(['player','position']).size().reset_index(name='counts')
wpm_grouped = wpm_grouped.sort_values(by = 'counts', ascending = False)
wpm_grouped

,player,position,counts
25,Rekkles,ADC,8
35,xiye,Middle,6
29,Uzi,ADC,6
7,Cody Sun,ADC,6
21,Padden,ADC,6
19,Mystic,ADC,5
31,Xiaohu,Middle,5
27,Scout,Middle,5
24,Pray,ADC,5
0,An,ADC,5


In [17]:
# Since Rekkles is our winner lets see which games those are
rekkles_filter = wpm_v_csm['player'] == 'Rekkles'
rekkles_games = wpm_v_csm[rekkles_filter]
rekkles_games.head()

,gameid,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
507,1002390211,ADC,Rekkles,Fnatic,Varus,Ezreal,Jarvan IV,Xayah,Syndra,Taliyah,...,9.090909,4097,4332,-235,6062,7161,-1099,3552,3486,66
572,1002380204,ADC,Rekkles,Fnatic,Twitch,Kalista,Jarvan IV,Sejuani,Renekton,Syndra,...,11.936967,4011,2608,1403,6156,4630,1526,3036,2422,614
723,1002390468,ADC,Rekkles,Fnatic,Tristana,Taliyah,Jarvan IV,Kog'Maw,Jayce,Trundle,...,9.440895,3337,3470,-133,4912,5609,-697,3237,3088,149
776,1002380323,ADC,Rekkles,Fnatic,Twitch,Kalista,Janna,Rakan,Taliyah,Galio,...,9.225941,3101,3281,-180,5392,6017,-625,2732,3006,-274
783,1002380330,ADC,Rekkles,Fnatic,Xayah,Gragas,Jarvan IV,Sejuani,Taliyah,Jayce,...,11.687968,3282,3526,-244,5541,6552,-1011,2889,3451,-562


In [18]:
# Lets sort by wpm and then look at the wins for those games
rekkles_sorted = rekkles_games.sort_values(by = 'wpm')
rekkles_sorted[['gameid','wpm','result','gdat10','gdat15']]

,gameid,wpm,result,gdat10,gdat15
776,1002380323,0.251046,0,-180,-625
831,1002390592,0.252708,1,-207,-175
848,1002390599,0.254925,1,1076,1288
812,1002380336,0.262870,1,296,1357
783,1002380330,0.264434,1,-244,-1011
572,1002380204,0.315167,0,1403,1526
507,1002390211,0.330579,0,-235,-1099
723,1002390468,0.335463,0,-133,-697


In [19]:
# Lets see if there are any correlations between how rekkles does early 
# and wards purchesed
rekkles_games[['wpm','result','gdat10','gdat15']].corr()

,wpm,result,gdat10,gdat15
wpm,1.000000,-0.714760,0.039152,-0.203794
result,-0.714760,1.000000,0.013411,0.278565
gdat10,0.039152,0.013411,1.000000,0.875236
gdat15,-0.203794,0.278565,0.875236,1.000000


In [20]:
# Looks like when rekkles buys less wards they win
# Could that be because those games were just against easy opponents
rek_ids = rekkles_games['gameid'].where(rekkles_games['result'] == 1)
rek_ids = list(rek_ids)
rek_ids = rek_ids[4:]
rek_opp = player_data[player_data['gameid'].isin(rek_ids)]


In [25]:
# Now that we have these lets see who the teams are
rek_opp = rek_opp[rek_opp.team != 'Fnatic']
rek_opp

,gameid,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,...,cspm,goldat10,oppgoldat10,gdat10,goldat15,oppgoldat15,gdat15,xpat10,oppxpat10,xpdat10
785,1002380330,Top,Flame,Immortals,Maokai,Kalista,Syndra,Rakan,Trundle,Shen,...,6.240635,3307,3113,194,5056,5180,-124,4552,4728,-176
786,1002380330,Jungle,Xmithie,Immortals,Ezreal,Kalista,Syndra,Rakan,Trundle,Shen,...,4.521816,2828,3396,-568,5195,5055,140,3365,2976,389
787,1002380330,Middle,Pobelter,Immortals,Orianna,Kalista,Syndra,Rakan,Trundle,Shen,...,7.483473,3308,3581,-273,5462,5386,76,4596,5065,-469
788,1002380330,ADC,Cody Sun,Immortals,Tristana,Kalista,Syndra,Rakan,Trundle,Shen,...,10.815337,3526,3282,244,6552,5541,1011,3451,2889,562
789,1002380330,Support,Olleh,Immortals,Lulu,Kalista,Syndra,Rakan,Trundle,Shen,...,0.661084,2248,2533,-285,3991,4247,-256,2863,3158,-295
804,1002380336,Top,Archie,Gigabyte Marines,Urgot,Janna,Tristana,Sejuani,Varus,Gnar,...,7.360350,2909,3623,-714,4828,5287,-459,3650,4338,-688
805,1002380336,Jungle,Levi,Gigabyte Marines,Kayn,Janna,Tristana,Sejuani,Varus,Gnar,...,4.403067,3035,3433,-398,4617,5163,-546,3648,3834,-186
806,1002380336,Middle,Optimus,Gigabyte Marines,Corki,Janna,Tristana,Sejuani,Varus,Gnar,...,7.820372,2969,3264,-295,4907,5349,-442,4610,4772,-162
807,1002380336,ADC,Noway,Gigabyte Marines,Xayah,Janna,Tristana,Sejuani,Varus,Gnar,...,9.857612,2970,3266,-296,4764,6121,-1357,2894,3090,-196
808,1002380336,Support,Sya,Gigabyte Marines,Lulu,Janna,Tristana,Sejuani,Varus,Gnar,...,0.230011,2352,2918,-566,3696,4941,-1245,2706,2855,-149
